In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_vals = pd.read_csv('../input/richters-predictor-modeling-earthquake-damage/train_values.csv')
train_labels = pd.read_csv('../input/richters-predictor-modeling-earthquake-damage/train_labels.csv')

In [ ]:
train_vals.head()

In [ ]:
train_vals.info()

In [ ]:
train_vals.dtypes.value_counts()

In [ ]:
train_df = train_vals.merge(train_labels, on = 'building_id')
train_df.head()

In [ ]:
cats = []
for column in train_df.columns:
        if train_df[column].dtype == 'object':
            cats.append(column)
for cat in cats:
    train_df[cat] = train_df[cat].astype('category')
len(cats)
cats

In [ ]:
corr = train_df.corr()['damage_grade'].sort_values(ascending = False)[1:]
corr = corr[abs(corr.values) > 0.01]

In [ ]:
plt.figure(figsize= (10,10))
sns.barplot(x = corr.index, y = corr, color = 'teal')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
for cat in cats:
    if cat in corr.index:
        print(cat)

In [ ]:
numericals = ['count_floors_pre_eq', 'height_percentage', 'area_percentage', 'count_families']

fig, axes = plt.subplots(2,2, figsize = (10,10))
for i, feature in enumerate(numericals):
    sns.histplot(data = train_df, x = feature, ax = axes[i%2, i//2], color = 'purple')
plt.show()

In [ ]:
sns.set_palette("colorblind")
fig, axes = plt.subplots(4,2, figsize = (20,20), sharex = False)
for i, name in enumerate(cats):
    sns.countplot(data = train_df, x = name, ax = axes[i%4, i//4])
plt.show()

In [ ]:
ls_dummies = pd.get_dummies(train_df['land_surface_condition'], prefix = 'ls_')
df_1 = pd.concat([train_df, ls_dummies], axis = 1)
df_1 = df_1.drop('land_surface_condition', axis = 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
for cat in cats:
    train_df[cat] = lb_make.fit_transform(train_df[cat])
train_df.head()

In [ ]:
train_df['damage_grade'].value_counts()

In [ ]:
X = train_df.drop('damage_grade', axis = 1)
y = train_df['damage_grade']

In [ ]:
test_vals = pd.read_csv('../input/richters-predictor-modeling-earthquake-damage/test_values.csv')

In [ ]:
for cat in cats:
    test_vals[cat] = lb_make.fit_transform(test_vals[cat])

In [ ]:
df_with_corrs = train_df.copy()
df_with_corrs = df_with_corrs.drop(columns = cats)

In [ ]:
x_corrs = df_with_corrs.drop('damage_grade', axis = 1)
y_corrs = df_with_corrs['damage_grade']

In [ ]:
test_vals = test_vals.drop(columns = cats)
test_vals

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dtree = DecisionTreeRegressor()
dtree.fit(x_corrs, y_corrs)

In [ ]:
y1 = dtree.predict(test_vals)
y1

In [ ]:
y2 = pd.Series(y1)
y2.value_counts()

In [ ]:
sns.countplot(y2)

In [ ]:
sns.countplot(train_df['damage_grade'])